In [1]:
import cogdl
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn.functional as F

In [2]:
import sys
sys.path.append('..')

# Load data

In [3]:
import dgl
import networkx
from grb.dataset.dataloader import DataLoader

Using backend: pytorch


In [4]:
dataset = DataLoader('cora')

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [5]:
dataset.adj

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 10556 stored elements in COOrdinate format>

In [6]:
dataset.adj_tensor

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)

# Load model

In [7]:
from grb.model.gcn import GCN

In [8]:
model = GCN(3, [1433, 64, 64, 7], activation=F.elu)

In [9]:
model.load_state_dict(torch.load('../grb/model/saved_models/model_gcn_cora.pt'))

<All keys matched successfully>

# Model training

In [10]:
from grb.model.trainer import Trainer

In [11]:
adam = torch.optim.Adam(model.parameters(), lr=0.01) 
nll_loss = F.nll_loss
device = 'cpu'
trainer = Trainer(dataset=dataset, optimizer=adam, loss=nll_loss, device=device)

In [12]:
trainer.set_config(n_epoch=100, eval_every=50, save_path='../grb/model/saved_models/model_gcn_cora.pt')

In [13]:
trainer.train(model)

Epoch 00000 | Loss 0.0000 | Train Acc 1.0000 | Val Acc 0.7020
Epoch 00050 | Loss 0.0000 | Train Acc 1.0000 | Val Acc 0.6420


# Evaluation

In [14]:
adj = dataset.adj
adj_tensor = dataset.adj_tensor
features = dataset.features
labels = dataset.labels

In [15]:
pred = model.forward(features, adj_tensor)

In [16]:
pred_label = torch.argmax(pred, dim=1)
pred_label.shape

torch.Size([2708])

In [17]:
from grb.utils import evaluator

In [18]:
acc = evaluator.eval_acc(pred, labels, mask=dataset.test_mask)
acc

tensor(0.6330)

# Attack

In [19]:
from grb.attack.speit import Speit

In [20]:
target_node = np.random.choice(np.arange(1000), 100)

In [21]:
config = {}
config['n_inject'] = 100
config['n_target_total'] = 1000
config['target_node'] = target_node
config['mode'] = 'random-inter'
config['lr'] = 0.01
config['feat_lim_min'] = 0
config['feat_lim_max'] = 1

In [22]:
speit = Speit(dataset, n_epoch=100, n_inject=100, n_edge_max=100)
speit.set_config(**config)

In [23]:
adj_attack = speit.injection(target_node, config['mode'])
adj_attack

<2808x2808 sparse matrix of type '<class 'numpy.float64'>'
	with 20610 stored elements in COOrdinate format>

In [24]:
features_attack = speit.attack(model, features, adj, target_node)

Epoch 0, Loss: 164.10159, Test acc: 0.91500
Epoch 1, Loss: -1177.22974, Test acc: 0.82400
Epoch 2, Loss: -4620.98584, Test acc: 0.80400
Epoch 3, Loss: -8569.72461, Test acc: 0.78500
Epoch 4, Loss: -12529.92773, Test acc: 0.77700
Epoch 5, Loss: -16685.82227, Test acc: 0.77200
Epoch 6, Loss: -21003.74023, Test acc: 0.76000
Epoch 7, Loss: -25322.08984, Test acc: 0.75100
Epoch 8, Loss: -29655.33398, Test acc: 0.74300
Epoch 9, Loss: -34026.46484, Test acc: 0.73800
Epoch 10, Loss: -38421.10547, Test acc: 0.73300
Epoch 11, Loss: -42834.43750, Test acc: 0.71800
Epoch 12, Loss: -47265.25391, Test acc: 0.70800
Epoch 13, Loss: -51717.11328, Test acc: 0.69500
Epoch 14, Loss: -56181.17578, Test acc: 0.68900
Epoch 15, Loss: -60660.65625, Test acc: 0.68000
Epoch 16, Loss: -65149.37500, Test acc: 0.65900
Epoch 17, Loss: -69651.74219, Test acc: 0.65000
Epoch 18, Loss: -74171.80469, Test acc: 0.64000
Epoch 19, Loss: -78754.14062, Test acc: 0.62700
Epoch 20, Loss: -83446.54688, Test acc: 0.60800
Epoch 21

In [25]:
speit.save_features(features_attack, './', 'features.npy')
speit.save_adj(adj_attack, './', 'adj.pkl')